В задаче необходимо для компаний с флагом -1 в поле okved2 файла inn_info_public проставить их отрасль. (Задача много-классовой классификации)

In [3]:
import pandas as pd
pays = pd.read_csv('pays.csv')
inn_info = pd.read_csv('inn_info_public.csv')

In [2]:
pays.head()

,hash_inn_kt,hash_inn_dt,week,count,sum
0,0,0,1,4,38399.6
1,0,0,5,2,399.6
2,0,0,7,2,79.6
3,0,0,9,2,239.6
4,0,0,12,2,79.6


In [4]:
inn_info.head()

,hash_inn,okved2,region,is_public
0,61058,34,86,True
1,8311,18,86,True
2,130273,-1,86,False
3,64081,43,86,True
4,218005,12,86,True
